In [ ]:
# !pip install deepeval
# !pip install python-dotenv
# !pip install rouge-score
# !pip install nltk


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=15b68232745607f03bff984a973ab32f607c8ba5855846a04fa6495a7c0a00ef
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import re
import numpy as np
from typing import List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Importing metrics from deepeval
from deepeval.metrics.ragas import (
    RAGASContextualPrecisionMetric,
    RAGASFaithfulnessMetric,
    RAGASContextualRecallMetric,
    RAGASAnswerRelevancyMetric
)
from deepeval.test_case import LLMTestCase
from deepeval.evaluate import evaluate

# Download necessary NLTK data files
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Import NLTK translation metrics
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

# Import ROUGE scorer
from rouge_score import rouge_scorer

## Evaluation Metrics Overview

This evaluation framework assesses our RAG system using both quantitative and qualitative metrics. The quantitative metrics provide objective, automated scores that reflect the similarity, consistency, and factuality of the generated answers. The qualitative metrics capture aspects such as context relevance, diversity, and clarity, which are essential for understanding the overall performance in real-world applications.

---

### Quantitative Metrics

- **BLEUMetric**  
  **What it measures:** Similarity between the generated answer and the expected output based on n-gram overlap.  
  **Explanation:** Uses NLTK's `sentence_bleu` to compute a BLEU score. A higher score indicates a better overlap with the reference answer.

- **ROUGEMetric**  
  **What it measures:** Recall-oriented overlap between the generated answer and the expected output.  
  **Explanation:** Computes ROUGE-1, ROUGE-2, and ROUGE-L F-measures and averages them. This reflects the degree to which key phrases and overall content are captured.

- **METEORMetric**  
  **What it measures:** Similarity between the generated answer and the expected output, addressing certain limitations of BLEU.  
  **Explanation:** Uses NLTK’s `meteor_score` to compute the METEOR score, providing an alternative perspective on answer quality.

- **FactualConsistencyMetric**  
  **What it measures:** The factual consistency between the generated answer and the retrieved context.  
  **Explanation:** Computes the cosine similarity between the TF-IDF vector representations of the generated answer and the combined retrieval context. A higher score indicates better alignment of the answer with the supporting context.

---

### Qualitative Metrics

- **RAGASContextualPrecisionMetric**  
  **What it measures:** Relevance of the retrieved context to the query.  
  **Threshold:** >0.7.

- **RAGASFaithfulnessMetric**  
  **What it measures:** Factual consistency between the generated answer and the retrieved context.  
  **Threshold:** >0.8.

- **RAGASContextualRecallMetric**  
  **What it measures:** Completeness of the retrieved context with respect to the query.  
  **Threshold:** >0.6.

- **RAGASAnswerRelevancyMetric**  
  **What it measures:** Relevance of the generated answer to the query.  
  **Threshold:** >0.7.

- **ContextualRelevancyMetric (Custom)**  
  **What it measures:** Combined relevance of the context and the generated answer to the query, computed via cosine similarity using TF-IDF vectors.  
  **Threshold:** >0.65.

- **ContextualDiversityMetric (Custom)**  
  **What it measures:** Diversity of the retrieved context.  
  **Threshold:** >0.5.

- **ContextualConsistencyMetric (Custom)**  
  **What it measures:** Consistency of the generated answer across multiple queries or context sources.  
  **Threshold:** >0.7.

- **TechnicalTermClarityMetric (Custom)**  
  **What it measures:** Clarity of technical term explanations in the generated answer.  
  **Threshold:** At least 1 explanation per technical term.

- **QuerySpecificityMetric (Custom)**  
  **What it measures:** Specificity of the generated answer relative to the query.  
  **Threshold:** >0.7.

- **CrossDocumentConsistencyMetric (Custom)**  
  **What it measures:** Consistency of the generated answer across multiple documents or context sources.  
  **Threshold:** >0.8.


In [ ]:
test_cases = [
    LLMTestCase(
        input="What is the role of attention mechanisms in Transformers?",
        actual_output="Attention mechanisms (components that weigh the importance of different input tokens) enable Transformers to process sequential data efficiently.",
        expected_output="Attention mechanisms allow Transformers to focus on relevant parts of the input.",
        context=[
            "Transformers use attention mechanisms to process sequential data. (Attention mechanisms are components that weigh the importance of different input tokens.)",
            "Attention was introduced in the 2017 'Attention Is All You Need' paper."
        ],
        retrieval_context=[
            "oAttentin mechanisms in Transformers...",
            "The 2017 paper introduced self-attention..."
        ]
    ),
    # Add more test cases
]

In [ ]:
class BLEUMetric:
    """
    BLEU Metric calculates the BLEU score between the generated answer (actual_output)
    and the expected output as a measure of similarity.
    
    A higher BLEU score indicates better overlap between the generated answer and the reference.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        reference = [test_case.expected_output.split()]
        hypothesis = test_case.actual_output.split()
        smoothing = SmoothingFunction().method1
        self.score = sentence_bleu(reference, hypothesis, smoothing_function=smoothing)
        return self.score

class ROUGEMetric:
    """
    ROUGE Metric calculates ROUGE scores (e.g., ROUGE-1, ROUGE-2, ROUGE-L) between the
    generated answer (actual_output) and the expected output.
    
    These scores reflect the recall-oriented overlap between the generated and reference texts.
    """
    def __init__(self):
        self.score = 0
        self.details = {}

    def measure(self, test_case: LLMTestCase) -> float:
        scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        scores = scorer.score(test_case.expected_output, test_case.actual_output)
        # For simplicity, we can average the three ROUGE scores (using F-measure)
        self.details = scores
        self.score = np.mean([score.fmeasure for score in scores.values()])
        return self.score

class METEORMetric:
    """
    METEOR Metric calculates the METEOR score between the generated answer (actual_output)
    and the expected output. METEOR is designed to address some weaknesses of BLEU.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        self.score = meteor_score([test_case.expected_output], test_case.actual_output)
        return self.score

class FactualConsistencyMetric:
    """
    FactualConsistencyMetric measures the factual consistency between the generated answer 
    and the retrieved context by computing the cosine similarity (using TF-IDF vectors)
    between the answer and the combined retrieval context.
    
    A higher score indicates a better alignment of the generated answer with the context.
    """
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        answer = test_case.actual_output
        context = " ".join(test_case.retrieval_context)
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([answer, context])
        similarity = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        self.score = similarity
        return self.score

In [4]:
class ContextualRelevancyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        query = test_case.input
        answer = test_case.actual_output
        context = " ".join(test_case.retrieval_context)
        
        # Compute similarity between query, answer, and context
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform([query, answer, context])
        query_answer_sim = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        query_context_sim = cosine_similarity(vectors[0:1], vectors[2:3])[0][0]
        
        self.score = (query_answer_sim + query_context_sim) / 2
        return self.score
    


class ContextualDiversityMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        
        # Compute pairwise similarity between contexts
        vectorizer = TfidfVectorizer()
        vectors = vectorizer.fit_transform(contexts)
        similarity_matrix = cosine_similarity(vectors)
        
        # Average similarity (lower = more diverse)
        avg_similarity = np.mean(similarity_matrix[np.triu_indices(len(contexts), 1)])
        self.score = 1 - avg_similarity
        return self.score
    

class ContextualConsistencyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        answer = test_case.actual_output
        
        # Check if answer appears in all contexts
        consistent = all([answer in ctx for ctx in contexts])
        self.score = 1.0 if consistent else 0.0
        return self.score
    

class TechnicalTermClarityMetric:
    def __init__(self):
        self.score = 0
        self.explanations = []

    def measure(self, test_case: LLMTestCase) -> float:
        answer = test_case.actual_output
        # Regex to detect terms with explanations in parentheses
        pattern = r"([A-Z][a-zA-Z\s]+) \((.+?)\)"
        matches = re.findall(pattern, answer)
        
        self.explanations = [term[0] for term in matches]
        self.score = len(matches) / len(self._detect_technical_terms(answer))
        return self.score

    def _detect_technical_terms(self, text: str) -> List[str]:
        # Replace with your domain-specific term list
        technical_terms = ["attention mechanisms", "Transformers", "self-attention"]
        return [term for term in technical_terms if term in text]
    

class QuerySpecificityMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        query = test_case.input
        answer = test_case.actual_output
        
        # Simple heuristic: Longer answers for specific queries
        query_words = len(query.split())
        answer_words = len(answer.split())
        self.score = min(answer_words / (query_words * 3), 1.0)
        return self.score
    

class CrossDocumentConsistencyMetric:
    def __init__(self):
        self.score = 0

    def measure(self, test_case: LLMTestCase) -> float:
        contexts = test_case.retrieval_context
        answer = test_case.actual_output
        
        # Check if answer appears in all contexts
        consistent = all([answer in ctx for ctx in contexts])
        self.score = 1.0 if consistent else 0.0
        return self.score
    


In [ ]:
def evaluate_rag_system(test_cases):
    metrics = [
        RAGASContextualPrecisionMetric(threshold=0.7),
        RAGASFaithfulnessMetric(threshold=0.8),
        RAGASContextualRecallMetric(threshold=0.6),
        RAGASAnswerRelevancyMetric(threshold=0.7),
        ContextualRelevancyMetric(),
        ContextualDiversityMetric(),
        ContextualConsistencyMetric(),
        TechnicalTermClarityMetric(),
        QuerySpecificityMetric(),
        CrossDocumentConsistencyMetric(),
        # New quantitative metrics:
        BLEUMetric(),
        ROUGEMetric(),
        METEORMetric(),
        FactualConsistencyMetric()
    ]
    
    return evaluate(test_cases, metrics)

In [ ]:
results = evaluate_rag_system(test_cases)
for metric in results:
    metric_name = metric.__class__.__name__
    # For ROUGEMetric, include additional details if available.
    if metric_name == "ROUGEMetric":
        print(f"{metric_name}: {metric.score:.3f} (Details: {metric.details}) (Passed: {metric.score >= 0.0})")
    else:
        print(f"{metric_name}: {metric.score:.3f} (Passed: {metric.score >= getattr(metric, 'threshold', 0)})")